In [142]:
import pandas as pd

url = "https://download-data.deutschebahn.com/static/datasets/haltestellen/D_Bahnhof_2020_alle.CSV"
df = pd.read_csv(url, sep=';')

#dropping Status column
df = df.drop(["Status"], axis = 1)

#dropping none values
df.dropna(inplace=True)

#dropping rows with empty values of Betreiber_Nr
df = df[df["Betreiber_Nr"] >= 0 ]

#dropping rows where the value in the "Verkehr" column is not 'FV', 'RV', or 'nur DPN'
df = df[df['Verkehr'].isin(['FV', 'RV', 'nur DPN'])]

#dropping rows where Lanege and Breite values are not between -90 and 90
df['Laenge'] = df['Laenge'].str.replace(',', '.')
df['Laenge'] = df['Laenge'].astype(float)
df = df[(df["Laenge"] <= 90) & (df["Laenge"] >= -90)]

df['Breite'] = df['Breite'].str.replace(',', '.')
df['Breite'] = df['Breite'].astype(float)
df = df[(df["Breite"] <= 90) & (df["Breite"] >= -90)]

#dropping rows where IFOPT has more than two letter in first pattern
df = df[df['IFOPT'].str[2] == ':']

In [134]:
from sqlalchemy import create_engine, BIGINT, TEXT, FLOAT

# Create a SQLAlchemy engine to connect to the SQLite database
engine = create_engine('sqlite:///trainstops.sqlite')

# Define the desired column types for the SQLite database
dtype = {
    'EVA_NR': BIGINT,
    'DS100': TEXT,
    'IFOPT': TEXT,
    'NAME': TEXT,
    'Verkehr': TEXT,
    'Laenge': FLOAT,
    'Breite': FLOAT,
    'Betreiber_Name': TEXT,
    'Betreiber_Nr': FLOAT
}

# Save the DataFrame to the SQLite database table named "trainstops"
df.to_sql('trainstops', engine, if_exists='replace', index=False, dtype=dtype)

# Close the SQLAlchemy engine
engine.dispose()

In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database file
conn = sqlite3.connect('trainstops.sqlite')

# Fetch data from the database using a SQL query and store it in a pandas DataFrame
query = 'SELECT * FROM trainstops'
df2 = pd.read_sql_query(query, conn)

for i in df2.columns:
    print(i, type(df2[i][0]))

conn.close()

EVA_NR <class 'numpy.int64'>
DS100 <class 'str'>
IFOPT <class 'str'>
NAME <class 'str'>
Verkehr <class 'str'>
Laenge <class 'numpy.float64'>
Breite <class 'numpy.float64'>
Betreiber_Name <class 'str'>
Betreiber_Nr <class 'numpy.float64'>
